sourced from: https://medium.com/vacatronics/3-ways-to-calibrate-your-camera-using-opencv-and-python-395528a51615

run before everything to callibrate camera

In [1]:
import cv2
import numpy as np
import pathlib
import os
import imutils
import cv2.aruco as aruco
from imutils.video import VideoStream
import time

In [7]:
def save_coefficients(mtx, dist, path):
    '''Save the camera matrix and the distortion coefficients to given path/file.'''
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()

def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

In [3]:

# Create gridboard, which is a set of Aruco markers
# the following call gets a board of markers 5 wide X 7 tall
gridboard = aruco.GridBoard_create(
        markersX=3, 
        markersY=3, 
        markerLength=0.1, 
        markerSeparation=0.01, 
        dictionary=aruco.Dictionary_get(aruco.DICT_4X4_50))

# Create an image from the gridboard
img = gridboard.draw(outSize=(900, 900))
cv2.imwrite("test_gridboard.jpg", img)

# Display the image to us
cv2.imshow('Gridboard', img)
# Exit on any key
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [8]:
def calibrate_aruco(dirpath, image_format, marker_length, marker_separation):
    '''Apply camera calibration using aruco.
    The dimensions are in cm.
    '''
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    arucoParams = aruco.DetectorParameters_create()
    board = aruco.GridBoard_create(3, 3, marker_length, marker_separation, aruco_dict)

    counter, corners_list, id_list = [], [], []
    img_dir = pathlib.Path(dirpath)
    first = 0
    # Find the ArUco markers inside each image
    for img in img_dir.glob(f'*.jpg'):
        print(str(img))
        image = cv2.imread(str(img))
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = aruco.detectMarkers(
            img_gray, 
            aruco_dict, 
            parameters=arucoParams
        )
        if first == 0:
            corners_list = corners
            id_list = ids
        else:
            corners_list = np.vstack((corners_list, corners))
            id_list = np.vstack((id_list,ids))
        first = first + 1
        counter.append(len(ids))

    counter = np.array(counter)
    # Actual calibration
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(
        corners_list, 
        id_list,
        counter, 
        board, 
        img_gray.shape, 
        None, 
        None 
    )
    return [ret, mtx, dist, rvecs, tvecs]

In [9]:

# Parameters
IMAGES_DIR = '/Users/ivyli/149proj/calib1214'
IMAGES_FORMAT = '.png'
# Dimensions in cm
MARKER_LENGTH = 5.5
MARKER_SEPARATION = 1.1

# Calibrate 
ret, mtx, dist, rvecs, tvecs = calibrate_aruco(
    IMAGES_DIR, 
    IMAGES_FORMAT,
    MARKER_LENGTH,
    MARKER_SEPARATION
)
# Save coefficients into a file
save_coefficients(mtx, dist, "calibration_aruco2.yml")

# Load coefficients
mtx, dist = load_coefficients('calibration_aruco1.yml')
original = cv2.imread('/Users/ivyli/149proj/sample33.jpg')
dst = cv2.undistort(original, mtx, dist, None, None)
cv2.imwrite('undist.jpg', dst)

/Users/ivyli/149proj/calib1214/gb12.jpg
/Users/ivyli/149proj/calib1214/gb13.jpg
/Users/ivyli/149proj/calib1214/gb11.jpg
/Users/ivyli/149proj/calib1214/gb10.jpg
/Users/ivyli/149proj/calib1214/gb28.jpg
/Users/ivyli/149proj/calib1214/gb14.jpg
/Users/ivyli/149proj/calib1214/gb15.jpg
/Users/ivyli/149proj/calib1214/gb29.jpg
/Users/ivyli/149proj/calib1214/gb17.jpg
/Users/ivyli/149proj/calib1214/gb16.jpg
/Users/ivyli/149proj/calib1214/gb7.jpg
/Users/ivyli/149proj/calib1214/gb6.jpg
/Users/ivyli/149proj/calib1214/gb4.jpg
/Users/ivyli/149proj/calib1214/gb5.jpg
/Users/ivyli/149proj/calib1214/gb2.jpg
/Users/ivyli/149proj/calib1214/gb3.jpg
/Users/ivyli/149proj/calib1214/gb8.jpg
/Users/ivyli/149proj/calib1214/gb9.jpg
/Users/ivyli/149proj/calib1214/gb27.jpg
/Users/ivyli/149proj/calib1214/gb26.jpg
/Users/ivyli/149proj/calib1214/gb24.jpg
/Users/ivyli/149proj/calib1214/gb30.jpg
/Users/ivyli/149proj/calib1214/gb18.jpg
/Users/ivyli/149proj/calib1214/gb19.jpg
/Users/ivyli/149proj/calib1214/gb25.jpg
/Users/i

True

In [11]:
mtx

array([[1.59213406e+03, 0.00000000e+00, 2.90901053e+02],
       [0.00000000e+00, 1.19010464e+03, 5.03743563e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [13]:
dist

array([[ 1.18030101, -2.18801478,  0.42753859,  0.06792766,  3.6625339 ]])

In [11]:
dist

array([[-0.05039761, -0.00940703,  0.03854019,  0.00064986,  0.00259853]])

In [12]:
mtx

array([[491.34073143,   0.        , 292.91190506],
       [  0.        , 466.80074034, 483.69311262],
       [  0.        ,   0.        ,   1.        ]])

In [2]:

vs = VideoStream(src=0).start()
while True:
    frame = vs.read()
    image = imutils.resize(frame, width = 1000)
    cv2.imshow('test', image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    

In [3]:
vs = VideoStream(src=0).start()
counter = 26
for i in range(5):
    input("press enter to take picture")
    frame = vs.read()
    image = imutils.resize(frame, width = 1000)
    cv2.imwrite("/Users/ivyli/149proj/calib1214/gb" + str(counter) + ".jpg", image)
    print('saved pic ' + str(counter))
    counter += 1

press enter to take picture
saved pic 26
press enter to take picture
saved pic 27
press enter to take picture
saved pic 28
press enter to take picture
saved pic 29
press enter to take picture
saved pic 30


In [3]:
vs = VideoStream(src=0).start()
frame = vs.read()
image = imutils.resize(frame, width = 1000)
cv2.imwrite("/Users/ivyli/149proj/sample33.jpg", image)

True

In [10]:
mtx, dist = load_coefficients('calibration_aruco2.yml')
original = cv2.imread('/Users/ivyli/149proj/sample33.jpg')
dst = cv2.undistort(original, mtx, dist, None, None)
cv2.imwrite('undist.jpg', dst)

True